# Notebook 3: Brand Insights & Report Generation

This notebook covers:
1. Brand sentiment comparison
2. Narrative theme extraction per brand
3. Retail channel attribution
4. Sentiment trends over time
5. Generating the full HTML report

> **Prerequisites:** `data/processed/annotated.parquet` must exist (run Notebook 2 or `make analyze`).

In [ ]:
import sys
sys.path.insert(0, '../src')

import pandas as pd
import plotly.express as px
import plotly.graph_objects as go
from pathlib import Path

In [ ]:
annotated_path = Path('../data/processed/annotated.parquet')
if annotated_path.exists():
    df = pd.read_parquet(annotated_path)
    print(f'Loaded {len(df):,} annotated records')
else:
    # Generate synthetic demo data
    import random
    random.seed(42)
    brands_pool = [['Nike'], ['Adidas'], ['New Balance'], ['Hoka'], ['Under Armour'],
                   ['Nike', 'Adidas'], ['Li-Ning'], ['Puma'], ['Asics'], []]
    channels_pool = [['StockX'], ['GOAT'], ['Nike Direct'], ['Foot Locker'],
                     ['StockX', 'GOAT'], [], ['Amazon'], ['Grailed']]
    intents = ['completed_purchase', 'seeking_purchase', 'price_discussion',
               'availability_info', 'purchase_consideration', None, None, None]
    
    rows = []
    for i in range(500):
        brands = random.choice(brands_pool)
        sentiment = random.gauss(0.15, 0.4)
        rows.append({
            'id': f'r{i}', 'subreddit': random.choice(['Sneakers','Nike','Adidas','Running']),
            'record_type': 'post' if i < 250 else 'comment',
            'score': random.randint(1, 500),
            'created_utc': pd.Timestamp('2024-01-01', tz='UTC') + pd.Timedelta(hours=i*3),
            'full_text': f'Sample text about sneakers and comfort quality hype #{i}',
            'vader_score': sentiment, 'hybrid_score': sentiment,
            'transformer_score': None,
            'brands': brands,
            'channels': random.choice(channels_pool),
            'primary_intent': random.choice(intents),
            'all_intents': [],
        })
    df = pd.DataFrame(rows)
    print(f'Using synthetic demo data: {len(df):,} records')

## 1. Brand Sentiment Comparison

In [ ]:
from reddit_sentiment.analysis.brand_comparison import BrandComparisonAnalyzer

analyzer = BrandComparisonAnalyzer()
metrics = analyzer.compute(df)
table = analyzer.comparison_table(df)

print(f'Brands detected: {list(metrics.keys())}')
display(table)

In [ ]:
# Brand sentiment bar chart
brands_sorted = table.sort_values('avg_sentiment', ascending=True)
colours = ['#22c55e' if s > 0.05 else '#ef4444' if s < -0.05 else '#94a3b8'
           for s in brands_sorted['avg_sentiment']]

fig = go.Figure(go.Bar(
    x=brands_sorted['avg_sentiment'],
    y=brands_sorted['brand'],
    orientation='h',
    marker_color=colours,
    text=[f"{s:+.3f}" for s in brands_sorted['avg_sentiment']],
    textposition='outside'
))
fig.update_layout(
    title='Brand Sentiment Comparison',
    xaxis_title='Avg. Hybrid Sentiment Score',
    xaxis=dict(range=[-1, 1]),
    height=450, plot_bgcolor='white'
)
fig.add_vline(x=0, line_dash='dash', line_color='gray')
fig.show()

## 2. Narrative Theme Analysis

In [ ]:
from reddit_sentiment.analysis.narrative import NarrativeThemeExtractor

extractor = NarrativeThemeExtractor()
narrative = extractor.extract(df)

theme_df = pd.DataFrame([
    {'theme': k, 'count': v, 'pct': narrative.theme_percentages.get(k, 0)}
    for k, v in sorted(narrative.theme_counts.items(), key=lambda x: x[1], reverse=True)
])
display(theme_df)

fig = px.bar(
    theme_df, x='pct', y='theme', orientation='h',
    title='Narrative Theme Frequency (% of corpus)',
    labels={'pct': '% of Posts', 'theme': 'Theme'}
)
fig.show()

In [ ]:
# Top TF-IDF terms
if narrative.top_tfidf_terms:
    print('Top TF-IDF terms:')
    print(', '.join(narrative.top_tfidf_terms[:25]))

## 3. Retail Channel Attribution

In [ ]:
from reddit_sentiment.analysis.channel_attribution import ChannelAttributionAnalyzer

ch_analyzer = ChannelAttributionAnalyzer()
attribution = ch_analyzer.analyze(df)

print('Channel Share:')
for ch, share in sorted(attribution.channel_share.items(), key=lambda x: x[1], reverse=True):
    print(f'  {ch}: {share:.1f}% ({attribution.channel_counts[ch]} mentions)')

# Pie chart
if attribution.channel_counts:
    ch_df = pd.DataFrame(list(attribution.channel_counts.items()), columns=['channel', 'count'])
    fig = px.pie(ch_df.head(8), values='count', names='channel',
                 title='Retail Channel Share', hole=0.3)
    fig.show()

## 4. Sentiment Trends

In [ ]:
from reddit_sentiment.analysis.trends import SentimentTrendAnalyzer

trend_analyzer = SentimentTrendAnalyzer()
trends = trend_analyzer.analyze(df)

if not trends.monthly.empty:
    fig = px.line(
        trends.monthly, x='period', y='avg_sentiment',
        title='Monthly Average Sentiment',
        markers=True
    )
    fig.add_hline(y=0, line_dash='dash', line_color='gray')
    fig.update_layout(yaxis=dict(range=[-1, 1]))
    fig.show()
    display(trends.monthly)

## 5. Generate Full HTML Report

In [ ]:
from reddit_sentiment.reporting.generator import ReportGenerator

reports_dir = Path('../data/reports')
generator = ReportGenerator(reports_dir=reports_dir)
html_path, md_path = generator.generate(df)

print(f'HTML Report: {html_path}')
print(f'Markdown:    {md_path}')
print('\nOpen the HTML report in your browser to see the full interactive dashboard!')